# Node
> This is the implementation of the Node class.

In [1]:
#|default_exp Node

In [6]:
#| hide
from nbdev.showdoc import *

In [9]:
#| export

import numpy as np
from alphaUltimateTTT.UltimateTicTacToe import UltimateTicTacToe as uttt
from alphaUltimateTTT.Move import Move
from alphaUltimateTTT.constants import *

In [10]:
#| export
#Copyright 2024 Gerardo Guerrero

class Node:
    def __init__(self,
                 game = uttt(),
                 args = None,
                 state = None,
                 parent = None,
                 action = None):
        self.game = game.copy()
        self.args = args
        self.state = state
        self.parent = parent
        self.action = action
        
        self.children = []
        self.visits = 0
        self.value = 0

        self.expandable_actions = self.game._get_legal_indexes()

    def is_fully_expanded(self):
        "Returns True if the node has no expandable actions, False otherwise."
        return len(self.expandable_actions) == 0
    
    def is_terminal(self):
        "Returns True if the nod terminal, False otherwise."
        return self.game.is_game_over()
    
    def select(self):
        "Selects the best child node according to the UCB formula."
        best_child = None
        best_ucb = -np.inf

        for child in self.children:
            ucb = self._get_ucb(child)
            if ucb > best_ucb:
                best_ucb = ucb
                best_child = child
        
        return best_child
    
    def expand(self):
        "Expands the node by adding a new child node."
        action = np.random.choice(self.expandable_actions)
        self.expandable_actions.remove(action)

        self.game._make_move(Move(self.game.next_symbol, action))
        next_state = self.game._get_state()
        child = Node(self.game, self.args, next_state, self, action)
        self.children.append(child)
        return child
    
    def backpropagate(self, value):
        "Updates the value and visits of the node and its ancestors."
        self.visits += 1

        # Check if the current node is a maximizing player (X) or a minimizing player (O)
        if self.game.next_symbol == X_STATE_VALUE:
            # If the current player is X, use the value as is
            self.value += value
        else:
            # If the current player is O, invert the value
            self.value -= value

        if self.parent is not None:
            # Propagate the value to the parent node with inversion if necessary
            self.parent.backpropagate(value)
    
    def simulate(self, player):
        "Simulates the game until the end."
        game = self.game.copy()

        while not game.is_game_over():
            legal_moves = game._get_legal_indexes()
            action = np.random.choice(legal_moves)
            game._make_move(Move(game.next_symbol, action))

        winner = game.result
        if winner == player:
            return 1
        elif winner == DRAW_STATE_VALUE:
            return 0
        else:
            return -1

    def _get_ucb(self, child):
        "Returns the UCB value for the given child."
        exploitation = 1 - ((child.value / child.visits) + 1)/2
        exploration = np.sqrt(np.log(self.visits) / child.visits)
        return exploitation + exploration